# Bank Customer Churn


## Data Background

This dataset can be found [here](https://www.kaggle.com/datasets/gauravtopre/bank-customer-churn-dataset) on kaggle.com.  It appears to be fictitious data created for a ML classification exercise

## Data Dictionary


* **customer_id** - unused variable.
* **credit_score** - used as input.
* **country** - used as input.
* **gender** - used as input.
* **age** - used as input.
* **tenure** - used as input.
* **balance** - used as input.
* **products_number** - used as input.
* **credit_card** - used as input.
* **active_member** - used as input.
* **estimated_salary** - used as input.
* **churn** - used as the target. 1 if the client has left the bank during some period or 0 if he/she has not.

## Imports

In [2]:
# Analysis, Visualization
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
# DBMS
import duckdb
# ML
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import (train_test_split, GridSearchCV,
                                     RandomizedSearchCV)
# Models
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
# Metrics
from sklearn.metrics import (mean_absolute_error, mean_squared_error, r2_score, 
accuracy_score, precision_score, recall_score, classification_report,
ConfusionMatrixDisplay)

## Database Creation

In [3]:
# creating connection to database
conn = duckdb.connect('churn_data.db')